In [11]:
import ollama

In [12]:
def generatellama(lol, hours):
  output=""
  ollama_response = ollama.chat(
    model='falcon:7b-text',
    stream=True,
    messages=[
      {
        'role': 'user',
        'content': lol,
      },
    ],
    options={
      'temperature': 0.4
    }
  )
  for chunk in ollama_response:
    if chunk['message']['content']=="\n":
      
      if hours==1:
        break
      hours-=1
    output+=chunk['message']['content']
  return output if output[0]!="\n" else output[1:]



In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

df = pd.read_csv('combined_IGI_airport.csv', parse_dates=['Timestamp'])
df.interpolate(method='linear', inplace=True)



In [14]:
import pandas as pd

df = pd.read_csv('combined_IGI_airport.csv', parse_dates=['Timestamp'])
df.interpolate(method='linear', inplace=True)


In [18]:
from io import StringIO
import tqdm

# Initialize lists to store metrics
actual_values_list = []
predicted_values_list = []

# Assuming df is your full dataframe, already preprocessed
for start_idx in tqdm.tqdm(range(0, len(df) - (168 + 24) + 1, 1000)):
    # Select chunks of data
    current_chunk = df.iloc[start_idx:start_idx + 168 + 24]
    input_df = current_chunk.head(168)
    actual_df = current_chunk.tail(24)
    
    # Prepare input and get predictions
    input_string = input_df.to_csv(index=False, header=False)
    predicted_values_string = generatellama(input_string, 24)  # Replace with actual call
    predicted_df = pd.read_csv(StringIO(predicted_values_string), header=None)
    predicted_df.columns = df.columns
    
    # Store actual and predicted values for metrics calculation
    actual_values_list.append(actual_df)
    predicted_values_list.append(predicted_df)

# Concatenate all actual and predicted values for metric calculations
all_actual_df = pd.concat(actual_values_list)
all_predicted_df = pd.concat(predicted_values_list)


  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def calculate_metrics(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    return mae, mse, rmse, mape

# Example calculation for one parameter - repeat for each and combine if needed
mae, mse, rmse, mape = calculate_metrics(all_actual_df['PM2.5 (µg/m³)'], all_predicted_df['PM2.5 (µg/m³)'])

print(f"Mean Absolute Error: {mae}\n Mean Squared Errpr: {mse}\n Root Mean Squared Error: {rmse}\n Mean Absolute Percentage Error: {mape}")
